### 과제 url

url => https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1

과제: 무신사 코디맵 페이지(많은 댓글 수 기준 정렬)에 나와있는 60개의 코디맵에 대하여 각각 ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers'] 정보를 가져와서 csv 형태로 만든 다음, 해당 코드(.ipynb)와 csv 파일을 본인 github 과제 repository에 제출

Challenge:
1. 각 코디맵마다 클릭한 뒤 다시 이전페이지로 돌아오는 코드를 반복문에 추가하여 ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl'] 정보도 가져오기
2.  page url을 format 함수를 이용해 수정한 뒤 반복문을 통해 1페이지부터 5페이지까지의 코디맵 정보 가져오기


In [30]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


In [31]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass


interval = 1 # 1초에 한번씩 스크롤 내림

# 현재 문서 높이를 가져와서 저장
prev_height = driver.execute_script("return document.body.scrollHeight")

# 반복 수행
while True:
    # 스크롤을 가장 아래로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(interval)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height



columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']         
category_lst=[]
title_lst=[]
date_lst=[]     
views_lst=[]   
comment_lst=[]    
                             
soup = BeautifulSoup(driver.page_source, 'lxml')        
data_rows = soup.find_all('li', attrs={'class':'style-list-item'}) 

for i, row in enumerate(data_rows):                     

    print('{}번째 코디정보 크롤링'.format(i+1))
    

    category = row.find('span', attrs={'class':'style-list-information__text'})       
    if category:                                                                   
        category = category.get_text().strip()                                          
        category_lst.append(category)
    else:                                                                      
        category_lst.append('Something is wrong')
        print('{}번째 codimap_category 가져올때 문제발생'.format(i+1))
        continue


    title = row.find('strong', attrs={'class':'style-list-information__title'})      
    if title:
        title = title.get_text().strip()
        title_lst.append(title)
    else:
        title_lst.append('Something is wrong')
        print('{}번째 codimap_title 가져올때 문제발생'.format(i+1))
        continue


    info = row.find_all('span', attrs={'class':'post-information__text'})  
    date_lst.append(info[0].get_text())
    views_lst.append(info[1].get_text())
    comment_lst.append(info[2].get_text())

    


    print('---------------------------------------------------')



df1 = pd.DataFrame(category_lst, columns = ['codimap_category'])
df2 = pd.DataFrame(title_lst, columns = ['codimap_title'])
df3 = pd.DataFrame(date_lst, columns = ['codimap_date'])
df4 = pd.DataFrame(views_lst, columns = ['views'])
df5 = pd.DataFrame(comment_lst, columns = ['comment_numbers'])
musinsa = pd.concat([df1,df2,df3,df4,df5],axis=1,join='inner')


1번째 코디정보 크롤링
---------------------------------------------------
2번째 코디정보 크롤링
---------------------------------------------------
3번째 코디정보 크롤링
---------------------------------------------------
4번째 코디정보 크롤링
---------------------------------------------------
5번째 코디정보 크롤링
---------------------------------------------------
6번째 코디정보 크롤링
---------------------------------------------------
7번째 코디정보 크롤링
---------------------------------------------------
8번째 코디정보 크롤링
---------------------------------------------------
9번째 코디정보 크롤링
---------------------------------------------------
10번째 코디정보 크롤링
---------------------------------------------------
11번째 코디정보 크롤링
---------------------------------------------------
12번째 코디정보 크롤링
---------------------------------------------------
13번째 코디정보 크롤링
---------------------------------------------------
14번째 코디정보 크롤링
---------------------------------------------------
15번째 코디정보 크롤링
---------------------------------------------------
16번째 코디정보 크롤링
-----

In [32]:
musinsa

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"조회 16,024",댓글 49
1,캐주얼,블랙 매니아,21.02.23,"조회 34,661",댓글 46
2,캐주얼,오늘의 코디,22.03.22,"조회 12,189",댓글 38
3,댄디,추천 댄디 룩,21.09.15,"조회 20,176",댓글 27
4,캐주얼,릴랙스 룩,21.02.23,"조회 23,528",댓글 22
5,스트릿,힙하다 힙해,21.02.03,"조회 35,278",댓글 21
6,캐주얼,심플한 코디,21.07.30,"조회 39,563",댓글 18
7,댄디,데이트 코디,21.12.24,"조회 10,302",댓글 18
8,댄디,훈훈 남친 룩,21.03.24,"조회 19,336",댓글 16
9,캐주얼,지금 입기 좋아,21.08.31,"조회 20,205",댓글 16


In [33]:
musinsa.to_csv('musinsa_basic.csv',index=False, encoding='utf-8-sig')

In [34]:
#기본 완료###########################################################

In [35]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

category_lst=[]
title_lst=[]
date_lst=[]     
views_lst=[]   
comment_lst=[]   
explain_lst=[]   
hashtag_lst=[]       
image_lst=[]

for page in range(1,6):

    interval = 1 
    prev_height = driver.execute_script("return document.body.scrollHeight")
    soup = BeautifulSoup(driver.page_source, 'lxml')         
    data_rows = soup.find_all('li', attrs={'class':'style-list-item'}) 


    # 반복 수행
    while True:
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height

    for i, row in enumerate(data_rows):                     

        print('{}번째 코디정보 크롤링'.format(i+1))

           
    

        category = row.find('span', attrs={'class':'style-list-information__text'})       
        if category:                                                                   
            category = category.get_text().strip()                                          
            category_lst.append(category)
        else:                                                                      
            category_lst.append('Something is wrong')
            print('{}번째 codimap_category 가져올때 문제발생'.format(i+1))
            continue


        title = row.find('strong', attrs={'class':'style-list-information__title'})      
        if title:
            title = title.get_text().strip()
            title_lst.append(title)
        else:
            title_lst.append('Something is wrong')
            print('{}번째 codimap_title 가져올때 문제발생'.format(i+1))
            continue


        info = row.find_all('span', attrs={'class':'post-information__text'})  
        date_lst.append(info[0].get_text())
        views_lst.append(info[1].get_text())
        comment_lst.append(info[2].get_text())


        # 상세 페이지 들어가기
        detail_button = driver.find_element(By.XPATH, '/html/body/div[3]/div[3]/form/div[4]/div/ul/li['+str(i+1)+']/div[1]/a')
        detail_button.click()
        time.sleep(1)

        soup2 = BeautifulSoup(driver.page_source, 'html.parser')

        explain =soup2.find('p', attrs={'class':'styling_txt'})
        if explain:
            explain= explain.get_text().strip()
            explain_lst.append(explain)        
        else:
            explain_lst.append('Something is wrong')
            print('codimap_explain 가져올때 문제발생')
            continue


        tag = soup2.find('div', attrs={'class':'ui-tag-list'})  
        if tag:
            tag= tag.get_text()
            hashtag_lst.append(tag)
        else:
            hashtag_lst.append('Something is wrong')
            print('codimap_hashtag 가져올때 문제발생')
            continue


        img = soup2.find('img', attrs={'class':'photo'})  
        if img:
            img = img['src']
            if img.startswith('//'):
                img = 'https:'+img
                image_lst.append(img)
        else:
            image_lst.append('Something is wrong')
            print('image_url 가져올때 문제발생')
            continue

        # 다시 나오기
        driver.back()

        




        print('---------------------------------------------------')
        
    page_button = driver.find_element(By.XPATH, '/html/body/div[3]/div[3]/form/div[4]/div/div[4]/div/div/a['+str(page+3)+']')
    page_button.click()
    time.sleep(1)


    df1 = pd.DataFrame(category_lst, columns = ['codimap_category'])
    df2 = pd.DataFrame(title_lst, columns = ['codimap_title'])
    df3 = pd.DataFrame(date_lst, columns = ['codimap_date'])
    df4 = pd.DataFrame(views_lst, columns = ['views'])
    df5 = pd.DataFrame(comment_lst, columns = ['comment_numbers'])
    df6 = pd.DataFrame(explain_lst, columns = ['codimap_explain'])
    df7 = pd.DataFrame(hashtag_lst, columns = ['codimap_hashtag'])
    df8 = pd.DataFrame(image_lst, columns = ['codimap_imgurl'])
    musinsa2 = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8],axis=1)
                       
           
  

   

1번째 코디정보 크롤링
---------------------------------------------------
2번째 코디정보 크롤링
---------------------------------------------------
3번째 코디정보 크롤링
---------------------------------------------------
4번째 코디정보 크롤링
---------------------------------------------------
5번째 코디정보 크롤링
---------------------------------------------------
6번째 코디정보 크롤링
---------------------------------------------------
7번째 코디정보 크롤링
---------------------------------------------------
8번째 코디정보 크롤링
---------------------------------------------------
9번째 코디정보 크롤링
---------------------------------------------------
10번째 코디정보 크롤링
---------------------------------------------------
11번째 코디정보 크롤링
---------------------------------------------------
12번째 코디정보 크롤링
---------------------------------------------------
13번째 코디정보 크롤링
---------------------------------------------------
14번째 코디정보 크롤링
---------------------------------------------------
15번째 코디정보 크롤링
---------------------------------------------------
16번째 코디정보 크롤링
-----

In [36]:
musinsa2

,codimap_category,codimap_title,codimap_date,views,comment_numbers,codimap_explain,codimap_hashtag,codimap_imgurl
0,캐주얼,톤온톤 코디,21.07.02,"조회 16,024",댓글 49,편하게 입기 좋은 반팔 티셔츠와 와이드 팬츠를 연출하고 크로스 백으로 마무리한 캐주얼 룩,\n#톤온톤\n#뉴트럴톤\n#캠퍼스\n#피크닉\n#와이드핏\n#오버핏\n#꾸안꾸\n...,https://image.msscdn.net/images/codimap/detail...
1,캐주얼,블랙 매니아,21.02.23,"조회 34,661",댓글 46,편안한 무드의 조거 팬츠에 블랙 스웨트셔츠를 매치한 후 볼캡으로 마무리한 캐주얼 룩,\n#봄\n#캐주얼\n#라퍼지스토어\n#다이아몬드 레이라\n#나이키\n#엠프렌즈\n...,https://image.msscdn.net/images/codimap/detail...
2,캐주얼,오늘의 코디,22.03.22,"조회 12,189",댓글 38,로고 디테일이 돋보이는 스웨트셔츠와 데님 팬츠를 코디하고 스니커즈로 완성한 캐주얼 룩,\n#로고플레이\n#오버핏\n#스티치\n#캠퍼스\n#봄\n#캐주얼\n#유니폼브릿지\...,https://image.msscdn.net/images/codimap/detail...
3,댄디,추천 댄디 룩,21.09.15,"조회 20,176",댓글 27,담백한 디테일의 니트와 셔츠를 레이어드하고 데님 팬츠를 더해 완성한 댄디 룩,\n#데이트\n#그래픽\n#레이어드룩\n#가을\n#댄디\n#엘무드\n#뮤지엄바이비컨...,https://image.msscdn.net/images/codimap/detail...
4,캐주얼,릴랙스 룩,21.02.23,"조회 23,528",댓글 22,플라워 자수가 눈에 띄는 니트에 조거 팬츠를 더한 후 레드 볼캡으로 컬러 포인트를 ...,\n#봄\n#캐주얼\n#카시오\n#코닥\n#제로\n#와릿이즌\n#뉴발란스\n#커버낫...,https://image.msscdn.net/images/codimap/detail...
...,...,...,...,...,...,...,...,...
295,스트릿,타이다이 코디,21.05.26,"조회 1,367",댓글 3,타이다이 패턴이 돋보이는 반팔 티셔츠에 스트링 팬츠를 코디하고 크로스 백으로 완성한...,\n#타이다이\n#유니크\n#봄\n#스트릿\n#논논\n#살로몬\n#디퍼런트이즈.오케...,https://image.msscdn.net/images/codimap/detail...
296,캐주얼,따라 입어봐,22.10.19,"조회 5,258",댓글 3,레터링이 돋보이는 후드 티셔츠와 데님 팬츠를 코디하고 스니커즈로 마무리한 캐주얼 룩,\n#레터링\n#와이드핏\n#오버핏\n#꾸안꾸\n#캠퍼스\n#리버서블\n#가을\n#...,https://image.msscdn.net/images/codimap/detail...
297,캐주얼,손이 가는 조합,22.09.19,"조회 6,058",댓글 3,블루 컬러가 매력적인 스웨트셔츠와 스타디움 재킷을 코디하고 데님 팬츠로 마무리한 캐...,\n#레터링\n#꾸안꾸\n#컬러블록\n#콜라보\n#가을\n#캐주얼\n#트래블\n#언...,https://image.msscdn.net/images/codimap/detail...
298,스트릿,힙하게 입어봐,22.10.11,"조회 3,775",댓글 3,자수 디테일이 돋보이는 스타디움 재킷과 데님 팬츠를 매치하고 스니커즈로 멋을 더한 ...,\n#모노톤\n#유니크\n#힙합\n#심볼\n#그래픽\n#벨루어\n#자수\n#컬러블록...,https://image.msscdn.net/images/codimap/detail...


In [37]:
musinsa2.to_csv('musinsa_challenge.csv',index=False, encoding='utf-8-sig')